In [ ]:
General Plan:
Querier asks for intersection data. For faster testing, we will just add 2 fake paths along with one true path for each of the infected patient.
Next, we will have 3 infected patient for this testing purpose. The server will send the big aggregated histogram of all infected patient to 
the first infected patient and the first one will decrease the fake counts from his histogram , the idea is that if some point is originally 
on his true path, he doesnot decrease the count on the histogram, if it does not exist on the path, he decreases the original count.

In [2]:
import collections
import hashlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt   
import math
import random
from collections import Counter


In [3]:
class TabulationHash:
    """Hash function for hashing by tabulation.

    The 32-bit key is split to four 8-bit parts. Each part indexes
    a separate table of 256 randomly generated values. Obtained values
    are XORed together.
    """

    def __init__(self, num_buckets):
        self.tables = [None] * 4
        for i in range(4):
            self.tables[i] = [random.randint(0, 0xffffffff) for _ in range(256)]
        self.num_buckets = num_buckets

    def hash(self, key):
        h0 = key & 0xff;
        h1 = (key >> 8) & 0xff;
        h2 = (key >> 16) & 0xff;
        h3 = (key >> 24) & 0xff;
        t = self.tables
        return (t[0][h0] ^ t[1][h1] ^ t[2][h2] ^ t[3][h3]) % self.num_buckets
    def binary_conv(self,num):
        mys = ''
        while(num>=1):
            mys+=str(num%2)
            num=int(num/2)
        rest_l = 32-len(mys)
        for i in range(rest_l):
            mys+='0'
        return mys[::-1]
            
    def converter(self,key,maximum_coordinate):
        '''This function converts a key to a 32 bit integer, so we are trying to make each co-ordinate 
        a 32 bit integer and the only way to do that is to map all tuples to unique integers and then map them using binary transformation.
        We are using integer coordinates only.'''
        int_num = key[0]*maximum_coordinate+key[1]
        #convert it to a binary integer and pad it with zeros for filling the previuos empty posts.
        bit_int = self.binary_conv(int_num)
        return self.hash(int(bit_int))

In [4]:
class Differential_Privacy_CMS():
    def __init__(self, width, depth,epsilon, max_coor):
        ''' Method to initialize the data structure
        @param width int: Width of the table
        @param depth int: Depth of the table (num of hash func)
        @param epsilon: privacy parameter
        @param max_coor: this is added for hash function
        In this case, when we declare differential privacy class instance, we give the parameters m and k, or width and depth
        '''
        self.width = width #hash values range between 0 to width-1
        self.depth = depth #number of hash functions
        self.epsilon = epsilon 
        self.tables = [None]*depth #going to be a 3D table where self.tables[i] is the ith hash function 
        self.max_coor = max_coor
        for i in range(depth):
            self.tables[i]= TabulationHash(width)
        
            
#Implementing the client side algorithm which works on each particular data element. First it selects an integer randomly from 0 to depth-1, 
#we store that value as j, next, we initialize a vector of -1, of length width, and set that jth index as 1. Then we create another vector of -1s
#and +1s, with fixed probabilities determined by the parameter epsilon. Then we do element wise multioplication and return the 
#result vector. 

    def CLient_Side(self,d):
        '''parameters are a data element, epsilon value and a hash family, but we are using mmh3 here so didn't add that as a parameter'''
        j = np.random.randint(self.depth)
        z_vect = np.zeros((1,self.width))
        v = z_vect-1 
        index = self.tables[j].converter(d,self.max_coor)
        v[0,index]=1
        val = np.exp(self.epsilon/2)
        probability_of_1 = val/(val+1) 
        probability_of_neg_1 = 1/(val+1)
        b = np.random.choice([1,-1],self.width,p=[probability_of_1,probability_of_neg_1]) 
        final_vector = v*b
        return (final_vector,j) 

    def Compute_Sketch_Matrix(self,D):
        '''So each element of D is a tuple where the first element is v_i and second element is j_i(which is created by the above funstion after getting 
        passed by Client_side each time), we have privacy parameter epsilon and dimensions. v_i is the a vector an it has the sma eshape as the 
        vectors returned by the Client_side algorithm'''
        val = np.exp(self.epsilon/2)
        n = len(D)
        c_epsilon = (val+1)/(val-1) 
        vec_one = np.ones((1,self.width))
        manipulated_data_matrix = np.zeros((n,self.width)) #Creatimg a matrix for x_is
        for elt in enumerate(D):
            new_vect = elt[1][0].reshape((1,self.width))
            man_vect = (c_epsilon/2)*new_vect #scalar and vector multiplication 
            half_vec_one = 0.5*vec_one 
            sum_vect = man_vect + half_vec_one
            manipulated_data_matrix[[elt[0]],:]= self.depth*sum_vect  
        M = np.zeros((self.depth,self.width))
        for elt in enumerate(D):
            for l in range(self.width): 
                M[elt[1][1],l]=M[elt[1][1],l]+manipulated_data_matrix[elt[0],l]
        return M 

    def Server_Side(self,Sketch_Matrix,d,length):
        '''It returns the estimated frequency of a data element given to it. So it has two parameters, data element and the length of the 
        data stream we are considering'''
        n = length 
        frac1 = self.width/(self.width-1)
        frac2 = n/self.width
        row_sum = 0
        for i in range(self.depth):
            index = self.tables[i].converter(d,self.max_coor)
            row_sum = row_sum + Sketch_Matrix[i,index]
        avg_row_sum = row_sum/self.depth 
        subtraction = avg_row_sum - frac2 
        assumed_freq = frac1*subtraction 
        return assumed_freq
        
    def Count_Mean_Sketch(self,D_s,D): 
        '''D_s is the stream of data and this is a subset of universe of data'''
        Modified_datalist = []
        Sketch_Matrix = []
        freq_vect = {}
        length = len(D_s) 
        for elt in enumerate(D_s):
            Modified_datalist.append(self.CLient_Side(elt[1])) 
        Sketch_Matrix = self.Compute_Sketch_Matrix(Modified_datalist)
        for d in D:
            freq_vect[d] = self.Server_Side(Sketch_Matrix,d,length) 
        #for d in freq_vect:
          
            #freq_vect[d]=round(freq_vect[d])
        return freq_vect
    

In [5]:
def frequency_counter_on_average(path_list,epsilon):
    '''This function runs the entire process multiple and take the running average of frequencies for reducing error'''
    u_list = [(x,y) for x in range(25) for y in range(25)]
    freq_counter_cum = {d:0 for d in u_list}
    for i in range(500):
        class_instance = Differential_Privacy_CMS(100,100,epsilon,24)
        freq_counter_new = class_instance.Count_Mean_Sketch(path_list, u_list)
        for d in freq_counter_new:
            freq_counter_cum[d]=(i*freq_counter_cum[d]+freq_counter_new[d])/(i+1)
    return freq_counter_cum

In [6]:
def dictionary_adder(dict1,dict2):
    z = dict1.copy()
    for elt in dict2:
        z[elt]=z.get(elt,0)+dict2[elt]
    return z

In [ ]:
def vertex_position_determiner(vertex):
    '''it determines whether any vertice is corner or on the border-edge or anything else'''
    x = vertex[0]
    y = vertex[1]
    if x==wrap.x_min or x==wrap.x_max:
        if y==wrap.y_min or y==wrap.y_max:
            return 'corner'
        else:
            return 'border-edge'
    elif y==wrap.y_min or y==wrap.y_max:
        return 'border-edge'
    else:
        return 'normal'
def corner_neighbor_gen(vertex):
    '''generates neighbors of 4 corners'''
    x = vertex[0]
    y = vertex[1]
    if x==wrap.x_min and y==wrap.y_min:
        return [(x+1,y),(x,y+1)]
    if x==wrap.x_max and y==wrap.y_min:
        return [(x-1,y),(x,y+1)] 
    if x==wrap.x_max and y==wrap.y_max:
        return [(x-1,y),(x,y-1)]
    if x==wrap.x_min and y==wrap.y_max:
        return [(x+1,y),(x,y-1)]
def side_neighbor_gen(vertex):    
    ''' generates neighbors of vertices on the side of the grid but not the corner'''
    x = vertex[0]
    y = vertex[1]
    if x==wrap.x_min:
        return [(x,y-1),(x,y+1),(x+1,y)]
    if x==wrap.x_max:
        return [(x-1,y),(x,y+1),(x,y-1)]
    if y==wrap.y_min:
        return [(x-1,y),(x+1,y),(x,y+1)] 
    if y==wrap.y_max:
        return [(x,y-1),(x-1,y),(x+1,y)] 
def normal_neighbor_gen(vertex):
    '''generates neighbors of all other vertices'''
    x = vertex[0]
    y = vertex[1]
    return [(x-1,y),(x+1,y),(x,y+1),(x,y-1)]
def path_creator(center_vertex,path_length):
    '''it will start creating a path of length given as a parameter in the function starting from the center vertex.'''
    current_vertex = center_vertex
    path = [] 
    for i in range(path_length):
        path.append(current_vertex)
        pos_det = vertex_position_determiner(current_vertex)
        if pos_det=='normal':
            ind = np.random.randint(4) 
            current_vertex = normal_neighbor_gen(current_vertex)[ind] 
        elif pos_det == 'corner': 
            ind = np.random.randint(2)
            current_vertex = corner_neighbor_gen(current_vertex)[ind]
        else:
            ind = np.random.randint(3) 
            current_vertex = side_neighbor_gen(current_vertex)[ind]
    return path
def wrap(x_min, x_max, y_min, y_max, center_vertex, path_length,number_of_fake_paths):
    wrap.x_min = x_min
    wrap.x_max = x_max
    wrap.y_min = y_min
    wrap.y_max = y_max
    return path_generator(center_vertex,number_of_fake_paths, path_length) 
def path_generator(center_vertex, number_of_fake_paths,path_length):
    '''Will generate the mentioned number of fake paths after generating a true path and then fixing the middle point and generating a bunch 
    of fake paths from this.'''
    true_path = path_creator(center_vertex, path_length)
    fake_path = []
    for i in range(number_of_fake_paths):
        fake_path.append(path_creator(center_vertex, path_length))
    return true_path, fake_path

In [7]:
def path_list_frequency_generator(path_list,epsilon):
    '''path_list is a list of lists. This function just generates frequency histograms'''
    result_list =[]
    agg_dict = {}
    for elt in path_list:
        my_dict = frequency_counter_on_average(elt,epsilon)
        result_list.append(my_dict)
        agg_dict = dictionary_adder(agg_dict,my_dict)
    return result_list, agg_dict
        

In [8]:
class querier():
    def __init__(self,true_path):
        self.true_path = true_path
    def checker(self,big_dict):
        result_list =[]
        s = set(self.true_path)
        for elt in s:
            if big_dict.get(elt,0)>=1:
                result_list.append(elt)
        return result_list


In [9]:
class infected():
    def __init__(self,true_path,fake_path,id,global_id):
        self.__true_path = true_path
        self.__fake_path = fake_path #list of lists
        self.cpy_path = [self.__true_path]+self.__fake_path
        self.id = id #will be fixed by some controller part
        self.histogram = None #server will give this data afterwards, this is just a list of dictionaries.
        self.global_id = global_id
      
    def decrementer(self, big_hist, participant_list):
        '''Each infected patient will call this method with the id of the next person unless his id is the global id in which case he just sends it 
        to the querier.'''
        true_path_histogram = self.histogram.pop(0)
        for hist in self.histogram:
            for elt in hist:
                if elt not in true_path_histogram and big_hist:
                    big_hist[elt]=big_hist[elt]-hist[elt]
        if self.id==self.global_id:
            return participant_list[-1].checker(big_hist)
        if self.id!=self.global_id:#sent to the next infected
            return participant_list[self.id+1].decrementer(big_hist, participant_list)
        
                                                

            

        

In [10]:
def new_mode(infected_patients,querier, epsilon):
    '''querier and infected_patients are class instances which will be passed by the testing part, here infected_patients is a list of infected instances.'''
    #the querier's data is never sent to the server.
    big_dict = {} # aggregation of all histograms
    for infected_patient in infected_patients:
        dict_list, agg_dict = path_list_frequency_generator(infected_patient.cpy_path,epsilon)
        infected_patient.histogram = dict_list
        big_dict = dictionary_adder(big_dict, agg_dict)
    #then these big dict is sent to the first infected patient patient and all the freq_histograms are sent to the infected patients.
    participant_list = infected_patients+[querier]
    return infected_patients[0].decrementer(big_dict,participant_list)
    
    


In [11]:
def accuracy_checker(predicted_path, original_path):
    '''Here we have two lists of intersections, one original and one predicted path.'''
    ps = set(predicted_path)
    os = set(original_path)
    common = ps & os
    false_signals = ps-common
    missed_signals = os-common
    return common,false_signals, missed_signals

In [ ]:
#Test type 1-No intersection
true_pathi1, fake_pathsi1 = wrap(0,19,20,79,(15,27),200,2)
true_pathi2, fake_pathsi2 = wrap(0,19,20,79,(18,31),200,2)
true_pathi3, fake_pathsi3 = wrap(0,19,20,79,(7,45),200,2)
querier_true_path,fake_path = wrap(0,19,0,19,(15,7),200,2)
infected1 = infected(true_pathi1, fake_pathsi1,1,3)
infected2 = infected(true_pathi2, fake_pathsi2,2,3)
infected3 = infected(true_pathi3, fake_pathsi3,3,3)
infected_patients = [infected1, infected2, infected3]
client = querier(querier_true_path)
predicted_path = new_mode(infected_patients, client,5))
original_path = set(querier_true_path) & set(true_pathi1+true_pathi2+true_pathi3)
print(accuracy_checker(predicted_path, original_path))

In [ ]:
#Test type 2- 1 intersection(but came from all three infected patients)
querier_true_path, fake_path = wrap(0,19,0,19,(15,7),200,2)
true_pathi1, fake_pathsi1 = wrap(0,19,20,79,(17,56),199,2)
true_pathi1, fake_pathsi2 = wrap(0,19,20,79,(14,45),199,2)
true_pathi1, fake_pathsi3 = wrap(0,19,20,79,(18,31),199,2)
true_pathi1+=[querier_true_path[0]]
true_pathi2+=[querier_true_path[0]]
true_pathi3+=[querier_true_path[0]]
infected1 = infected(true_pathi1, fake_pathsi1,1,3)
infected2 = infected(true_pathi2, fake_pathsi2,2,3)
infected3 = infected(true_pathi3, fake_pathsi3,3,3)
infected_patients = [infected1, infected2, infected3]
client = querier(querier_true_path)
predicted_path = new_mode(infected_patients, client,5))
original_path = set(querier_true_path) & set(true_pathi1+true_pathi2+true_pathi3)
print(accuracy_checker(predicted_path, original_path))

In [ ]:
#Test type 3- 3 different intersections, from came from three infected patients.
querier_true_path, fake_path = wrap(0,19,0,19,(15,7),200,2)
true_pathi1, fake_pathsi1 = wrap(0,19,20,79,(17,56),199,2)
true_pathi1, fake_pathsi2 = wrap(0,19,20,79,(14,45),199,2)
true_pathi1, fake_pathsi3 = wrap(0,19,20,79,(18,31),199,2)
true_pathi1+=[querier_true_path[0]]
true_pathi2+=[querier_true_path[1]]
true_pathi3+=[querier_true_path[2]]
infected1 = infected(true_pathi1, fake_pathsi1,1,3)
infected2 = infected(true_pathi2, fake_pathsi2,2,3)
infected3 = infected(true_pathi3, fake_pathsi3,3,3)
infected_patients = [infected1, infected2, infected3]
client = querier(querier_true_path)
predicted_path = new_mode(infected_patients, client,5))
original_path = set(querier_true_path) & set(true_pathi1+true_pathi2+true_pathi3)
print(accuracy_checker(predicted_path, original_path))

In [ ]:
#Test type 4, multiple intersections, came from all infected patients.
true_pathi1, fake_pathsi1 = wrap(0,19,0,79,(11,45),200,2)
true_pathi2, fake_pathsi2 = wrap(0,19,0,79,(56,11),200,2)
true_pathi3, fake_pathsi3 = wrap(0,19,0,79,(7,4),200,2)
infected1 = infected(true_pathi1, fake_pathsi1,1,3)
infected2 = infected(true_pathi2, fake_pathsi2,2,3)
infected3 = infected(true_pathi3, fake_pathsi3,3,3)
infected_patients = [infected1, infected2, infected3]
querier_true_path, fake_path = wrap(0,19,0,79,(15,7),200,2)
client = querier(querier_true_path)
predicted_path = new_mode(infected_patients, client,5))
original_path = set(querier_true_path) & set(true_pathi1+true_pathi2+true_pathi3)
print(accuracy_checker(predicted_path, original_path))